# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
pip install folium

   ---------------------------------------- 0.0/108.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/108.4 kB ? eta -:--:--
   ---------------------- ---------------- 61.4/108.4 kB 812.7 kB/s eta 0:00:01
   -------------------------------------- 108.4/108.4 kB 892.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [37]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [39]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,22.54,71,85,0.45,US,1722975772
1,1,talnakh,69.4865,88.3972,9.50,72,10,4.36,RU,1722976295
2,2,petropavlovsk-kamchatsky,53.0452,158.6483,11.33,100,75,2.00,RU,1722976295
3,3,laredo,27.5064,-99.5075,39.16,30,0,4.63,US,1722976153
4,4,bredasdorp,-34.5322,20.0403,9.69,90,0,0.96,ZA,1722976295


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    color="City",
    alpha= 0.5,
    tiles= "ESRI",
    scale= 1
)

m.save('city_map.html')

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [43]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,calvi,42.5000,8.8333,23.35,78,0,1.03,FR,1722976299
76,76,taiyuan,37.8694,112.5603,24.92,69,0,2.00,CN,1722976301
86,86,nova vicosa,-17.8919,-39.3719,22.69,73,0,3.80,BR,1722976303
152,152,sokcho,38.2083,128.5911,26.34,83,0,0.51,KR,1722976309
160,160,es castell,39.8795,4.2854,26.38,73,0,1.54,ES,1722976310
191,191,brookings,44.3114,-96.7984,22.88,59,0,3.60,US,1722976210
192,192,megalopoli,37.4011,22.1422,22.25,62,0,1.32,GR,1722976313
211,211,faraskur,31.3298,31.7151,25.78,85,0,2.29,EG,1722976315
238,238,kasserine,35.1676,8.8365,25.39,49,0,3.00,TN,1722976318
261,261,leua,48.2156,18.6071,21.94,57,0,1.83,SK,1722976320


### Step 3: Create a new DataFrame called `hotel_df`.

In [45]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
46,calvi,FR,42.5000,8.8333,78,
76,taiyuan,CN,37.8694,112.5603,69,
86,nova vicosa,BR,-17.8919,-39.3719,73,
152,sokcho,KR,38.2083,128.5911,83,
160,es castell,ES,39.8795,4.2854,73,
191,brookings,US,44.3114,-96.7984,59,
192,megalopoli,GR,37.4011,22.1422,62,
211,faraskur,EG,31.3298,31.7151,85,
238,kasserine,TN,35.1676,8.8365,49,
261,leua,SK,48.2156,18.6071,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [47]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
calvi - nearest hotel: Bel horizon
taiyuan - nearest hotel: 汾酒大厦
nova vicosa - nearest hotel: Pousada Pontal da Barra
sokcho - nearest hotel: Happy House Hotel
es castell - nearest hotel: Hotel Agamenon
brookings - nearest hotel: Quality Inn
megalopoli - nearest hotel: No hotel found
faraskur - nearest hotel: No hotel found
kasserine - nearest hotel: Hôtel Saif
leua - nearest hotel: Levi Dom
manziana - nearest hotel: Agriturismo Sciambreto
kolonia - nearest hotel: Wasserturm Hotel Cologne
portland - nearest hotel: the Hoxton
adilcevaz - nearest hotel: Hotel Park
colonia - nearest hotel: Wasserturm Hotel Cologne
puyang chengguanzhen - nearest hotel: 贝壳酒店
le poinconnet - nearest hotel: Hôtel Kyriad
pamandzi - nearest hotel: Gite Le Madina
wuhai - nearest hotel: Binhe International Hotel
el kala - nearest hotel: فندق لالة فاطمة
mashhad - nearest hotel: هتل تعاون
sao fidelis - nearest hotel: Hotel São José
goias - nearest hotel: No hotel found
buka - nearest hotel: No

,City,Country,Lat,Lng,Humidity,Hotel Name
46,calvi,FR,42.5000,8.8333,78,Bel horizon
76,taiyuan,CN,37.8694,112.5603,69,汾酒大厦
86,nova vicosa,BR,-17.8919,-39.3719,73,Pousada Pontal da Barra
152,sokcho,KR,38.2083,128.5911,83,Happy House Hotel
160,es castell,ES,39.8795,4.2854,73,Hotel Agamenon
191,brookings,US,44.3114,-96.7984,59,Quality Inn
192,megalopoli,GR,37.4011,22.1422,62,No hotel found
211,faraskur,EG,31.3298,31.7151,85,No hotel found
238,kasserine,TN,35.1676,8.8365,49,Hôtel Saif
261,leua,SK,48.2156,18.6071,57,Levi Dom


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot

map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=1,
    color="City",
    alpha= 0.5,
    tiles="ESRI",
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)